In [2]:


import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import os


image_dir = r'C:\Users\marce\Downloads\phase2_train_v0\final'
labels_csv = r'C:\Users\marce\Downloads\phase2_train_v0.csv'


image_size = (128, 128) 
batch_size = 32


df = pd.read_csv(labels_csv)
class_names = df['class_label'].unique()
class_indices = {class_name: idx for idx, class_name in enumerate(class_names)}


def custom_image_generator(image_dir, df, batch_size, image_size):
    while True:
        batch_paths = np.random.choice(df['img_fName'], size=batch_size)
        batch_input = []
        batch_output = []
        
        for image_path in batch_paths:
            img = load_img(os.path.join(image_dir, image_path), target_size=image_size)
            img = img_to_array(img) / 255.0
            batch_input.append(img)
   
            label = df[df['img_fName'] == image_path]['class_label'].values[0]
            batch_output.append(to_categorical(class_indices[label], num_classes=len(class_names)))
        
        yield (np.array(batch_input), np.array(batch_output))

# Cargar el modelo ResNet50 preentrenado sin la capa superior
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)  # Capa de pooling global
x = Dense(256, activation='relu')(x)  # Capa fully connected con 256 neuronas
x = Dropout(0.5)(x)  # Añadir dropout para regularización
predictions = Dense(len(class_names), activation='softmax')(x)  # Capa de salida 


model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers[:-10]:
    layer.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True


optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)


model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


train_generator = custom_image_generator(image_dir, df, batch_size, image_size)


epochs = 40  
steps_per_epoch = len(df) // batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs
)

model.save('mosquito_classifier_resnet50.h5')

Epoch 1/40
205/323 [==================>...........] - ETA: 2:20 - loss: 1.1165 - accuracy: 0.4838

c:\Users\marce\AppData\Local\Programs\Python\Python37\lib\site-packages\PIL\Image.py:3179: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


323/323 [==============================] - 392s 1s/step - loss: 1.1117 - accuracy: 0.4841
Epoch 2/40
323/323 [==============================] - 380s 1s/step - loss: 1.0827 - accuracy: 0.5149
Epoch 3/40
323/323 [==============================] - 382s 1s/step - loss: 1.0641 - accuracy: 0.5293
Epoch 4/40
323/323 [==============================] - 381s 1s/step - loss: 1.0565 - accuracy: 0.5516
Epoch 5/40
323/323 [==============================] - 377s 1s/step - loss: 1.0417 - accuracy: 0.5536
Epoch 6/40
323/323 [==============================] - 379s 1s/step - loss: 1.0302 - accuracy: 0.5699
Epoch 7/40
323/323 [==============================] - 393s 1s/step - loss: 1.0217 - accuracy: 0.5771
Epoch 8/40
323/323 [==============================] - 391s 1s/step - loss: 1.0235 - accuracy: 0.5867
Epoch 9/40
323/323 [==============================] - 6961s 22s/step - loss: 1.0004 - accuracy: 0.5855
Epoch 10/40
323/323 [==============================] - 561s 2s/step - loss: 0.9900 - accuracy: 0.589

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os

model = tf.keras.models.load_model('mosquito_classifier_resnet50.h5')
image_path = r'C:\Users\marce\Downloads\phase2_train_v0\final\train_10338.jpeg'
image_size = (128, 128)


img = load_img(image_path, target_size=image_size)
img_array = img_to_array(img) / 255.0 
img_array = np.expand_dims(img_array, axis=0)  


predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
class_name = list(class_indices.keys())[list(class_indices.values()).index(predicted_class)]

print(f"Predicción: {class_name}")


1/1 [==============================] - 1s 622ms/step
Predicción: japonicus/koreicus


In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import os

# Ruta a las imágenes y archivo CSV
image_dir = r'C:\Users\marce\Downloads\phase2_train_v0\final'
labels_csv = r'C:\Users\marce\Downloads\phase2_train_v0.csv'


image_size = (128, 128)
batch_size = 32

df = pd.read_csv(labels_csv)
class_names = df['class_label'].unique()
class_indices = {class_name: idx for idx, class_name in enumerate(class_names)}


def custom_image_generator(image_dir, df, batch_size, image_size):
    while True:
        batch_paths = np.random.choice(df['img_fName'], size=batch_size)
        batch_input = []
        batch_output = []
        
        for image_path in batch_paths:
            img = load_img(os.path.join(image_dir, image_path), target_size=image_size)
            img = img_to_array(img) / 255.0  # Normalizar la imagen
            batch_input.append(img)
   
            label = df[df['img_fName'] == image_path]['class_label'].values[0]
            batch_output.append(tf.keras.utils.to_categorical(class_indices[label], num_classes=len(class_names)))
        
        yield (np.array(batch_input), np.array(batch_output))


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Añadir capas adicionales
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Capa de pooling global
x = Dense(256, activation='relu')(x)  # Capa fully connected con 256 neuronas
x = Dropout(0.5)(x) 
predictions = Dense(len(class_names), activation='softmax')(x) 

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Descongelar las últimas 30 capas para el ajuste fino
for layer in base_model.layers[:-30]:
    layer.trainable = False


optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
train_generator = custom_image_generator(image_dir, df, batch_size, image_size)


epochs = 200 
steps_per_epoch = 100 // batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs
)


model.save('mosquito_classifier_mobilenetv2.h5')

true_labels = []
predicted_labels = []
df_limited = df.sample(n=100, random_state=42)  # Seleccionar 100 imágenes aleatorias




Epoch 1/200
3/3 [==============================] - 6s 1s/step - loss: 2.5058 - accuracy: 0.1458
Epoch 2/200
3/3 [==============================] - 4s 1s/step - loss: 2.0516 - accuracy: 0.2396
Epoch 3/200
3/3 [==============================] - 3s 1s/step - loss: 1.3355 - accuracy: 0.4896
Epoch 4/200
3/3 [==============================] - 3s 1s/step - loss: 1.4162 - accuracy: 0.4688
Epoch 5/200
3/3 [==============================] - 4s 1s/step - loss: 1.4514 - accuracy: 0.4479
Epoch 6/200
3/3 [==============================] - 3s 1s/step - loss: 1.2988 - accuracy: 0.4896
Epoch 7/200
3/3 [==============================] - 4s 1s/step - loss: 1.5098 - accuracy: 0.4375
Epoch 8/200
3/3 [==============================] - 3s 1s/step - loss: 1.3105 - accuracy: 0.5208
Epoch 9/200
3/3 [==============================] - 3s 1s/step - loss: 1.3598 - accuracy: 0.5208
Epoch 10/200
3/3 [==============================] - 4s 1s/step - loss: 1.1266 - accuracy: 0.5833
Epoch 11/200
3/3 [=====================

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import os

# Ruta a las imágenes y archivo CSV
image_dir = r'C:\Users\marce\Downloads\phase2_train_v0\final'
labels_csv = r'C:\Users\marce\Downloads\phase2_train_v0.csv'

# Tamaño de las imágenes y batch
image_size = (128, 128)

# Leer archivo CSV que contiene las etiquetas
df = pd.read_csv(labels_csv)
class_names = df['class_label'].unique()
class_indices = {class_name: idx for idx, class_name in enumerate(class_names)}

# Definir el generador para cargar todas las imágenes del conjunto de prueba
def all_test_image_generator(image_dir, df, image_size):
    batch_input = []
    batch_output = []
    
    for image_path in df['img_fName']:
        img = load_img(os.path.join(image_dir, image_path), target_size=image_size)
        img = img_to_array(img) / 255.0  # Normalizar la imagen
        batch_input.append(img)
        
        # Obtener la etiqueta correcta
        label = df[df['img_fName'] == image_path]['class_label'].values[0]
        batch_output.append(class_indices[label])
    
    return np.array(batch_input), np.array(batch_output)

# Generar imágenes y etiquetas verdaderas
test_images, true_labels = all_test_image_generator(image_dir, df, image_size)

# Cargar el modelo previamente entrenado
model = tf.keras.models.load_model('mosquito_classifier_mobilenetv2.h5')

# Realizar predicciones con el modelo en todo el conjunto de datos
predictions = model.predict(test_images)

# Convertir las predicciones a índices de clase
predicted_labels = np.argmax(predictions, axis=1)

# Calcular la precisión
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Graficar la matriz de confusión
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


c:\Users\marce\AppData\Local\Programs\Python\Python37\lib\site-packages\PIL\Image.py:3179: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


OSError: No file or directory found at mosquito_classifier_mobilenetv2.h5